In [ ]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import l2beat_utils as ltwo
sys.path.pop()

import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
# # Get token classification
# # https://dune.com/queries/3852319
token_df = d.get_dune_data(query_id = 3852319,
    name = "tokens_by_type_base_opm",
    performance = 'large',
    num_hours_to_rerun=0
)

token_df.sample(5)

In [ ]:
# Get Lastest L2B aoc
l2b_df = ltwo.get_daily_aoc_by_token()
l2b_df_spc = l2b_df[l2b_df['project'].isin(['optimism','base'])]
l2b_df_spc = l2b_df_spc.rename(columns={'project':'blockchain'})

l2b_df_spc = l2b_df_spc[['dt','blockchain','token_type','asset_id','address','usd_value']]
l2b_df_spc['address'] = l2b_df_spc['address'].str.lower()

l2b_df_spc.sample(5)

In [ ]:
cols = ['blockchain','token_address','l1_token','token_classification','symbol']
token_if_l1_df = token_df[cols].rename(columns={'l1_token':'address'})
token_if_l2_df = token_df[cols].rename(columns={'token_address':'address'})

In [ ]:
token_if_l1_df = token_if_l1_df[~token_if_l1_df['address'].isna()]
token_if_l2_df = token_if_l2_df[~token_if_l2_df['address'].isna()]
token_if_l2_df.sample(5)

In [ ]:
unified_df = l2b_df_spc.merge(token_if_l1_df[token_if_l1_df['address'].isna()][['blockchain','address','token_classification']]
                              , on=['blockchain','address'], how='left')
unified_df = unified_df.merge(token_if_l2_df[['blockchain','address','token_classification']]
                              , on=['blockchain','address'], how='left')

In [ ]:
unified_df['token_classification'] = unified_df['token_classification_x'].fillna(unified_df['token_classification_y']).fillna('Unknown')
unified_df = unified_df.drop(columns=['token_classification_x', 'token_classification_y'])

In [ ]:
#Drop dupes where 1 L1 maps to multiple L2 tokens
unified_df = unified_df.drop_duplicates(keep='first')
unified_df.to_csv('csv_outputs/aoc_by_chain_token_classification.csv')

In [ ]:
unified_df.sample(5)

In [ ]:
grouped_df = unified_df.groupby(['blockchain', 'token_type', 'token_classification'])['usd_value'].sum().reset_index()

# Calculate the total usd_value for each blockchain
total_by_blockchain = grouped_df.groupby('blockchain')['usd_value'].sum().reset_index()
total_by_blockchain = total_by_blockchain.rename(columns={'usd_value': 'total_usd_value_by_blockchain'})
# Merge the total_by_blockchain back into the grouped_df
grouped_df = grouped_df.merge(total_by_blockchain, on='blockchain')
# Calculate the overall total usd_value
overall_total = grouped_df['usd_value'].sum()
# Add new columns for the percentages
grouped_df['percent_of_blockchain'] = (grouped_df['usd_value'] / grouped_df['total_usd_value_by_blockchain']) * 100
grouped_df['percent_of_overall'] = (grouped_df['usd_value'] / overall_total) * 100
# Drop the total_usd_value_by_blockchain column as it's no longer needed
grouped_df = grouped_df.drop(columns=['total_usd_value_by_blockchain'])
# Display the updated DataFrame
# print(grouped_df)
# # Display the grouped table
grouped_df.sort_values(by='usd_value',ascending=False)

In [ ]:
# Convert usd_value to millions
grouped_df['usd_value_millions'] = grouped_df['usd_value'] / 1e6

# Set the order of token_type
token_type_order = ['native', 'canonical', 'external']
grouped_df['token_type'] = pd.Categorical(grouped_df['token_type'], categories=token_type_order, ordered=True)

# Set the style and figure size
sns.set(style="whitegrid")
plt.figure(figsize=(20, 5 * len(token_type_order)))

# Create a custom color palette
color_palette = {'optimism': '#ff0420', 'base': '#0b45ed'}

# Create the faceted bar chart
g = sns.FacetGrid(grouped_df, row="token_type", height=4, aspect=2, sharex=True, sharey=True)

# Define the plotting function
def plot_bars(data, **kwargs):
    ax = sns.barplot(x="token_classification", y="usd_value_millions", hue="blockchain", data=data, 
                     palette=color_palette, hue_order=['optimism', 'base'], **kwargs)
    for p in ax.patches:
        ax.annotate(f'${p.get_height():.1f}M', (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), 
                    textcoords='offset points')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("USD Value (Millions)")

# Apply the plotting function to each facet
g.map_dataframe(plot_bars)

# Adjust the y-axis to be the same for all facets
y_max = grouped_df['usd_value_millions'].max()
g.set(ylim=(0, y_max))

# Add labels and title
g.set_axis_labels("Token Classification", "")
g.fig.suptitle("USD Value by Token Classification, Blockchain, and Token Type", fontsize=16, y=1.02)

# Adjust the legend
g.add_legend(title="Blockchain", bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()